In [1]:
import pandas as pd
import os
from myst_nb import glue

notebook_name = '3-creating-inputs.ipynb' # __file__ doesn't work in a jupyter notebook

input_dir = 'data/dcGO_input/downloaded'
HP_loc = os.path.join(input_dir, 'Domain2HP.txt')
MESH_loc = os.path.join(input_dir, 'Domain2CD.txt')
DO_loc = os.path.join(input_dir, 'Domain2DO.txt')
GO_loc = os.path.join(input_dir, 'Domain2GO_supported_only_by_all.txt') # high coverage
MP_loc = os.path.join(input_dir, 'Domain2MP.txt')

new_col_names = ['domain_type', 
                 'domain_sunid', 
                 'ont_term_id', 
                 'ont_term_name', 
                 'ont_subontologies', 
                 'information_content', 
                 'annotation_origin_1direct_0inherited']

hp = pd.read_csv(HP_loc, sep='\t', comment= '#')
hp.columns = new_col_names
hp['ont_id'] = 'HP'

mesh = pd.read_csv(MESH_loc, sep='\t', comment= '#')
mesh.columns = new_col_names
mesh['ont_id'] = 'MESH'

do = pd.read_csv(DO_loc, sep='\t', comment= '#')
do.columns = new_col_names
do['ont_id'] = 'DO'

go = pd.read_csv(GO_loc, sep='\t', comment= '#')
go.columns = new_col_names
go = go[go['ont_subontologies'] == 'biological_process']
go['ont_id'] = 'GO_BP'

mp = pd.read_csv(MP_loc, sep='\t', comment= '#')
mp.columns = new_col_names
mp['ont_id'] = 'MP'

all_onts = pd.concat([hp, mesh, do, go, mp], ignore_index=True)
outfile = 'data/dcGO_input/created/human_po.txt'
with open(outfile, 'w') as f:
    f.write(f'# file created at {pd.Timestamp.now()} by {notebook_name}.\n')
    f.write('# file contains high coverage mappings for HP, MESH (CD), DO, and GOBP.\n')
    all_onts.to_csv(f, index=False)
    
display(all_onts.head())

def glue_counts_dcGO(df: pd.DataFrame, identifier: str):
    """
    Expects a DataFrame df that contains only ontology terms of interest.
    """
    glue(f"{identifier}_assignments", len(df))
    glue(f"{identifier}_terms", len(df['ont_term_id'].unique()))
    glue(f"{identifier}_domains", len(df['domain_sunid'].unique()))
    
    return None
    
glue_counts_dcGO(all_onts[all_onts['ont_id']=='HP'], 'HP')
glue_counts_dcGO(all_onts[all_onts['ont_id']=='MESH'], 'MESH')
glue_counts_dcGO(all_onts[all_onts['ont_id']=='DO'], 'DO')
glue_counts_dcGO(all_onts[all_onts['ont_id']=='GO_BP'], 'GOBP')
glue_counts_dcGO(all_onts[all_onts['ont_id']=='MP'], 'MP')
glue_counts_dcGO(all_onts, 'all')

,domain_type,domain_sunid,ont_term_id,ont_term_name,ont_subontologies,information_content,annotation_origin_1direct_0inherited,ont_id
0,fa,57736,HP:0011420,Age of death,Mortality_Aging,0.0,0,HP
1,fa,47502,HP:0011420,Age of death,Mortality_Aging,0.0,0,HP
2,fa,47547,HP:0011420,Age of death,Mortality_Aging,0.0,0,HP
3,fa,90258,HP:0011420,Age of death,Mortality_Aging,0.0,0,HP
4,fa,57736,HP:0001699,Sudden death,Mortality_Aging,0.0,1,HP


15984

1978

562

5761

...

2528

719

357555

15286

3146

In [2]:
from IPython.display import HTML 

lines_to_view = [8000,20000,50000]
view = HTML(all_onts.iloc[lines_to_view].to_html(index=False)) 
glue("excerpt-pofile", view)

domain_type,domain_sunid,ont_term_id,ont_term_name,ont_subontologies,information_content,annotation_origin_1direct_0inherited,ont_id
fa,55528,HP:0000925,Abnormality of the vertebral column,Phenotypic_abnormality,1.123852,0,HP
sf,53822,MESH:D013568,"Pathological Conditions, Signs and Symptoms",CTD_diseases,0.581857,0,MESH
sf,48619,GO:0006658,phosphatidylserine metabolic process,biological_process,2.596963,1,GO_BP


In [3]:
import tabix
import os
import numpy as np

# Read in background and make a note of which calls are thre
background_loc = 'data/background.vcf'
bg_df = pd.read_csv(background_loc, 
                    usecols=['#CHROM', 'POS', 'REF', 'ALT'],
                    dtype={"#CHROM":str, 'POS':int, 'REF':str, 'ALT':str},
                    sep='\t', 
                    index_col=['#CHROM', 'POS'])
display(bg_df)

consequence_loc = 'data/consequences.tsv'
c_columns = ['#CHROM', 'POS', 'calls', 'snp_id', 'ENSP_id', 'prot_sub', 'HMM', 'position', 'ref_prob', 'mut_prob', 'SUPERFAMILY', 'Sup_e_val', 'FAMILY', 'Fam_e_val']

# TODO: Get combined 23andMe (v2 chip) and CAGI6 list
c_tb = tabix.open(f"{consequence_loc}.gz")
flipped_count = 0
selected_rows = []
for chrom, pos in bg_df.index.unique():
    results = c_tb.query(chrom, pos-1, pos)    
    ref, alt = bg_df.loc[(chrom, pos)]
    
    for result in results:
        call = result[2]
        if call == f"{ref}/{alt}":
            selected_rows.append(result)
        elif call == f"{alt}/{ref}":
            selected_rows.append(result)
            flipped_count+=1
print(flipped_count)
            
c_df = pd.DataFrame(selected_rows, columns=c_columns)
# Some versions of snowflake keep in information about proteins that don't have domain assignments (this removes those rows)
c_df = c_df.replace(r'^\s*$', np.nan, regex=True).dropna(subset=['FAMILY','SUPERFAMILY'])
c_df.to_csv('data/consequences_2500G.tsv', index=False)

c_df.set_index(['#CHROM', 'POS'], inplace=True)

REF ALT
#CHROM POS             
1      69224      A   T
       69428      T   G
       69487      G   A
       69496      G   A
       69521      T   A
...              ..  ..
Y      21897579   A   G
       21901440   G   C
       21906413   G   A
       22941490   G   A
       22942878   A   G

[561906 rows x 2 columns]

0


In [4]:
glue('consequence-rows', len(c_df.index))
glue('consequence-unique-snps', len(c_df.index.unique()))
glue('consequence-unique-proteins', len(c_df['ENSP_id'].unique())) 
glue('consequence-unique-families', len(c_df['FAMILY'].unique())) 
glue('consequence-unique-supfam', len(c_df['SUPERFAMILY'].unique()))

regs_snp = c_df.iloc[0].name
multi_family_snp = False
multi_supfam_snp = False
for chrom, pos in c_df[c_df.index.duplicated(keep=False)].index.unique():
    families_per_snp = len(c_df.loc[(chrom, pos)]['FAMILY'].unique())
    superfamilies_per_snp = len(c_df.loc[(chrom, pos)]['SUPERFAMILY'].unique())
    if families_per_snp > 1 and superfamilies_per_snp == 1:
        multi_family_snp = (chrom, pos)
    elif superfamilies_per_snp > 1:
        multi_supfam_snp = (chrom, pos)
        
    if multi_family_snp and multi_supfam_snp:
        break

index_to_view = [regs_snp, multi_family_snp, multi_supfam_snp]
glue("excerpt-consequence", c_df.loc[index_to_view])

503670

218534

38242

1859

1036

/Users/nataliethurlby/phd/jupyter_books_etc/jekyll-version/phenotype_from_genotype/venv/lib/python3.7/site-packages/ipykernel_launcher.py:11: PerformanceWarning: indexing past lexsort depth may impact performance.
  # This is added back by InteractiveShellApp.init_path()
/Users/nataliethurlby/phd/jupyter_books_etc/jekyll-version/phenotype_from_genotype/venv/lib/python3.7/site-packages/ipykernel_launcher.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  if sys.path[0] == '':


calls  snp_id          ENSP_id prot_sub        HMM position  \
#CHROM POS                                                                   
1      69224     A/T     NaN  ENSP00000334393     D45V  SF0037432       44   
       1290695   G/C     NaN  ENSP00000307887    T136S  SF0042359       95   
       1290695   G/C     NaN  ENSP00000344998     T35S  SF0047556       61   
       1290695   G/C     NaN  ENSP00000399229    T136S  SF0042359       95   
       3392588   T/C     NaN  ENSP00000367629    Y479H  SF0040099        5   
       3392588   T/C     NaN  ENSP00000367629    Y479H  SF0050917        5   
       3392588   T/C     NaN  ENSP00000408887    Y183H  SF0040099        5   
       3392588   T/C     NaN  ENSP00000408887    Y183H  SF0050917        5   

                      ref_prob mut_prob SUPERFAMILY Sup_e_val FAMILY Fam_e_val  
#CHROM POS                                                                      
1      69224    0.524940000001  4.26011       81321  2.93e-78  81320    0.0021  
       1290695         1.83872  2.56474       48726  3.14e-10  48727    0.0099  
       1290695         1.72866  2.45993       48726  0.000485  49159     0.077  
       1290695         1.83872  2.56474       48726  3.29e-10  48727    0.0099  
       3392588         2.69907   4.0507       50729  3.73e-18  50730     0.017  
       3392588         2.69907   4.0507       48065  3.53e-57  48066  8.09e-05  
       3392588         2.69907   4.0507       50729  1.32e-18  50730     0.017  
       3392588         2.69907   4.0507       48065  3.27e-38  48066   0.00025

In [5]:
# TODO: tabix consequence file
# TODO: filter background VCF file
display(c_df.loc[index_to_view])
# TODO: tabix VCF file

calls  snp_id          ENSP_id prot_sub        HMM position  \
#CHROM POS                                                                   
1      69224     A/T     NaN  ENSP00000334393     D45V  SF0037432       44   
       1290695   G/C     NaN  ENSP00000307887    T136S  SF0042359       95   
       1290695   G/C     NaN  ENSP00000344998     T35S  SF0047556       61   
       1290695   G/C     NaN  ENSP00000399229    T136S  SF0042359       95   
       3392588   T/C     NaN  ENSP00000367629    Y479H  SF0040099        5   
       3392588   T/C     NaN  ENSP00000367629    Y479H  SF0050917        5   
       3392588   T/C     NaN  ENSP00000408887    Y183H  SF0040099        5   
       3392588   T/C     NaN  ENSP00000408887    Y183H  SF0050917        5   

                      ref_prob mut_prob SUPERFAMILY Sup_e_val FAMILY Fam_e_val  
#CHROM POS                                                                      
1      69224    0.524940000001  4.26011       81321  2.93e-78  81320    0.0021  
       1290695         1.83872  2.56474       48726  3.14e-10  48727    0.0099  
       1290695         1.72866  2.45993       48726  0.000485  49159     0.077  
       1290695         1.83872  2.56474       48726  3.29e-10  48727    0.0099  
       3392588         2.69907   4.0507       50729  3.73e-18  50730     0.017  
       3392588         2.69907   4.0507       48065  3.53e-57  48066  8.09e-05  
       3392588         2.69907   4.0507       50729  1.32e-18  50730     0.017  
       3392588         2.69907   4.0507       48065  3.27e-38  48066   0.00025